In [1]:
library(reshape2)  # required to melt the correlation matrix
library(ggplot2)
library(corrplot)

corrplot 0.84 loaded


In [2]:
#!devtools::install_github("WinVector/WVPlots")

In [3]:
library(WVPlots)

- An Introduction to `corrplot` package: https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html
- Example `WVPlots`: https://cran.r-project.org/web/packages/WVPlots/vignettes/WVPlots_examples.html

In [4]:
df <- read.csv("mlbootcamp5_train.csv", sep=';')

In [5]:
head(df)

id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62,110,80,1,1,0,0,1,0
1,20228,1,156,85,140,90,3,1,0,0,1,1
2,18857,1,165,64,130,70,3,1,0,0,0,1
3,17623,2,169,82,150,100,1,1,0,0,1,1
4,17474,1,156,56,100,60,1,1,0,0,0,0
8,21914,1,151,67,120,80,2,2,0,0,0,0


### 1. Визуализируем корреляционную матрицу

* Для того чтобы лучше понять признаки в датасете, можно посчитать матрицу коэффициентов корреляции между признаками.

#### Какие два признака больше всего коррелируют (по Пирсону) с признаком `height` ?

- Gluc, Cholesterol
- Weight, Alco
- Smoke, Alco
- <b>Weight, Gender</b>

In [6]:
d <- round(cor(df, method='pearson'), 3)

In [7]:
head(d)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,1.000,0.003,0.004,-0.003,-0.002,0.003,-0.003,0.006,0.002,-0.004,0.001,0.004,0.004
age,0.003,1.000,-0.023,-0.082,0.054,0.021,0.018,0.154,0.099,-0.048,-0.030,-0.010,0.238
gender,0.004,-0.023,1.000,0.499,0.155,0.006,0.015,-0.036,-0.020,0.338,0.171,0.006,0.008
height,-0.003,-0.082,0.499,1.000,0.291,0.005,0.006,-0.050,-0.019,0.188,0.094,-0.007,-0.011
weight,-0.002,0.054,0.155,0.291,1.000,0.031,0.044,0.142,0.107,0.068,0.067,-0.017,0.182
ap_hi,0.003,0.021,0.006,0.005,0.031,1.000,0.016,0.024,0.012,-0.001,0.001,0.000,0.054


In [8]:
colors <- colorRampPalette(c("#67001F", "#B2182B", "#D6604D", "#F4A582",
                             "#FDDBC7", "#FFFFFF", "#D1E5F0", "#92C5DE",
                             "#4393C3", "#2166AC", "#053061"))

In [9]:
corrplot(d, method='square',
         tl.col = "black", cl.ratio = 0.3, number.cex=0.5,
         addCoefasPercent = TRUE,
         col = colors(100), bg = "white")

# Weight, Gender

---

### 2. Распределение роста для мужчин и женщин
* Как мы увидели, в процессе исследования уникальных значений пол кодируется значениями 1 и 2, расшифровка изначально не была нам дана в описании данных, но мы догадались, кто есть кто, посчитав средние значения роста (или веса) при разных значениях признака gender. Теперь сделаем то же самое, но графически.

#### Постройте violinplot для роста и пола. Используйте:

* hue – для разбивки по полу
* scale – для оценки количества каждого из полов

In [10]:
# geom_split_violin function

GeomSplitViolin <- ggproto("GeomSplitViolin", GeomViolin, draw_group = function(self, data, ..., draw_quantiles = NULL){
  data <- transform(data, xminv = x - violinwidth * (x - xmin), xmaxv = x + violinwidth * (xmax - x))
  grp <- data[1,'group']
  newdata <- plyr::arrange(transform(data, x = if(grp%%2==1) xminv else xmaxv), if(grp%%2==1) y else -y)
  newdata <- rbind(newdata[1, ], newdata, newdata[nrow(newdata), ], newdata[1, ])
  newdata[c(1,nrow(newdata)-1,nrow(newdata)), 'x'] <- round(newdata[1, 'x']) 
  if (length(draw_quantiles) > 0 & !scales::zero_range(range(data$y))) {
    stopifnot(all(draw_quantiles >= 0), all(draw_quantiles <= 
                                              1))
    quantiles <- ggplot2:::create_quantile_segment_frame(data, draw_quantiles)
    aesthetics <- data[rep(1, nrow(quantiles)), setdiff(names(data), c("x", "y")), drop = FALSE]
    aesthetics$alpha <- rep(1, nrow(quantiles))
    both <- cbind(quantiles, aesthetics)
    quantile_grob <- GeomPath$draw_panel(both, ...)
    ggplot2:::ggname("geom_split_violin", grid::grobTree(GeomPolygon$draw_panel(newdata, ...), quantile_grob))
  }
  else {
    ggplot2:::ggname("geom_split_violin", GeomPolygon$draw_panel(newdata, ...))
  }
})

geom_split_violin <- function (mapping = NULL, data = NULL, stat = "ydensity", position = "identity", ...,
                               draw_quantiles = NULL, trim = TRUE, scale = "area", na.rm = FALSE, show.legend = NA,
                               inherit.aes = TRUE) {
    layer(data = data,
          mapping = mapping,
          stat = stat,
          geom = GeomSplitViolin,
          position = position,
          show.legend = show.legend,
          inherit.aes = inherit.aes,
          params = list(trim = trim, scale = scale, draw_quantiles = draw_quantiles, na.rm = na.rm, ...))
}

In [11]:
ggplot(df, aes(x = factor(gender), y = height, fill = factor(gender))) + 
  geom_split_violin(scale='count')

* Постройте на одном графике два отдельных kdeplot роста, отдельно для мужчин и женщин. На нем разница будет более наглядной, но нельзя будет оценить количество мужчин/женщин.

In [12]:
ggplot(data=df, aes(x = height, fill = factor(gender))) + 
  geom_density(alpha = 0.3)  # alpha is transparency

---

### 3. Ранговая корреляция
* В большинстве случаев достаточно воспользоваться линейным коэффициентом корреляции Пирсона для выявления закономерностей в данных, но мы пойдём чуть дальше и используем ранговую корреляцию, которая поможет нам выявить пары, в которых меньший ранг из вариационного ряда одного признака всегда предшествует большему другого (или наоборот, в случае отрицательной корреляции).

Постройте корреляционную матрицу, используя коэффициент Спирмена.
#### 3.1 Какие признаки теперь больше всего коррелируют (по Спирмену) друг с другом?
 * Height, Weight
 * Age, Weight
 * <b>Ap_hi, Ap_lo</b>
 * Cholesterol, Gluc
 * Cardio, Cholesterol
 * Smoke, Alco

In [13]:
dt <- round(cor(df, method='spearman'), 3)

In [14]:
dt

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,1.000,0.003,0.004,-0.002,-0.001,0.003,0.000,0.006,0.003,-0.004,0.001,0.004,0.004
age,0.003,1.000,-0.020,-0.082,0.062,0.219,0.157,0.140,0.092,-0.047,-0.030,-0.010,0.234
gender,0.004,-0.020,1.000,0.534,0.172,0.063,0.066,-0.036,-0.019,0.338,0.171,0.006,0.008
height,-0.002,-0.082,0.534,1.000,0.314,0.019,0.031,-0.059,-0.024,0.198,0.097,-0.007,-0.012
weight,-0.001,0.062,0.172,0.314,1.000,0.278,0.254,0.138,0.110,0.072,0.068,-0.015,0.183
ap_hi,0.003,0.219,0.063,0.019,0.278,1.000,0.735,0.209,0.105,0.029,0.033,-0.001,0.452
ap_lo,0.000,0.157,0.066,0.031,0.254,0.735,1.000,0.167,0.082,0.026,0.039,0.003,0.363
cholesterol,0.006,0.140,-0.036,-0.059,0.138,0.209,0.167,1.000,0.408,0.015,0.041,0.008,0.215
gluc,0.003,0.092,-0.019,-0.024,0.110,0.105,0.082,0.408,1.000,0.000,0.019,-0.009,0.091
smoke,-0.004,-0.047,0.338,0.198,0.072,0.029,0.026,0.015,0.000,1.000,0.340,0.026,-0.015


In [15]:
corrplot(dt, method='number',
         tl.col = "black", cl.ratio = 0.3, number.cex=0.55,
         addCoefasPercent = TRUE,
         col = colors(100), bg = "white")

# Ap_hi, Ap_lo

#### 3.2 Почему мы получили такое большое (относительно) значение ранговой корреляции у этих признаков?

- Неточности в данных (ошибки при сборе данных)
- Связь ошибочна, переменные никак не должны быть связаны друг с другом
- <b>Природа данных</b>

---

### 4. Совместное распределение признаков
* Постройте совместный график распределения jointplot двух наиболее коррелирующих между собой признаков (по Спирмену).

* Кажется, наш график получился неинформативным из-за выбросов в значениях. Постройте тот же график, но с логарифмической шкалой  (чтобы не получать OverflowError необходимо отфильтровать значения меньше либо равные нулю).

In [16]:
data <- subset(df, df$ap_hi > 0 & df$ap_lo > 0)

In [17]:
nrow(data)

[1] 69971

In [18]:
data$ap_hi <- log(data$ap_hi)
data$ap_lo <- log(data$ap_lo)

In [19]:
head(data, 3)

id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62,4.700480,4.382027,1,1,0,0,1,0
1,20228,1,156,85,4.941642,4.499810,3,1,0,0,1,1
2,18857,1,165,64,4.867534,4.248495,3,1,0,0,0,1


In [20]:
# ScatterHist(data, xvar="ap_hi", yvar="ap_lo", title="title")

# works, result here.

<img src="R2_t4.jpg">

* 4.1 Сколько чётко выраженных кластеров получилось на совместном графике выбранных признаков, с логарифмической шкалой? Под кластером в данной задаче понимается плотное скопление точек, в окрестности которого пренебрежительно мало одиночных наблюдений и которое визуально отделимо от других кластеров.
  * 1
  * 2
  * 3
  * <b>больше трёх</b>

---

### 5. Возраст
* Посчитаем, сколько полных лет было респондентам на момент их занесения в базу.

In [21]:
df$age_years <- as.integer(df$age / 365.25)
head(df, 2)

id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years
0,18393,2,168,62,110,80,1,1,0,0,1,0,50
1,20228,1,156,85,140,90,3,1,0,0,1,1,55


* Постройте Countplot, где на оси абсцисс будет отмечен возраст, на оси ординат – количество. Каждое значение возраста должно     иметь два столбца, соответствующих количеству человек каждого класса cardio (здоров/болен) данного возраста.

#### 5. В каком возрасте количество пациентов с ССЗ  впервые становится больше, чем здоровых?
- 44
- 49
- <b>53</b>
- 62

In [22]:
centered <- subset(df, df$age_years > 48 & df$age_years < 60)

In [23]:
counts <- table(centered$cardio, centered$age_years)
barplot(counts, main="the thing",
        xlab="age", ylab="count", col=c("blue", "darkgreen"),
        legend=rownames(counts), beside=TRUE)